# Demand prediction baseline solution

Victor Kantor, xead.wl@gmail.com

In [22]:
import pandas as pd

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [23]:
frac = 0.1 # fraction of learning examples used for model fitting

# sample the train set if your don't want to deel with all examples
train = train.sample(frac=frac, random_state=42)

X = train.drop(['Num','y'], axis=1)
y = train['y']
print len(X), len(y)

7246 7246


In [24]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=43)
model.fit(X, y)

preds = model.predict(test.drop(['Num'], axis=1))

print len(preds)
print len(sample_submission)

2016
2016
CPU times: user 6.63 s, sys: 64.9 ms, total: 6.69 s
Wall time: 6.8 s


In [25]:
def metric(x, y):
    return 2 * np.sum(np.abs(x - y) / (np.abs(x) + np.abs(y))) / x.shape[0]

In [26]:
def smape_score(estimator, X, y):
    return metric(estimator.predict(X), y)

In [27]:
import sklearn.model_selection as ms

In [28]:
len(X), len(y)

(7246, 7246)

In [29]:
import numpy as np

In [30]:
ms.cross_val_score(model, X, y, scoring=smape_score)

array([ 0.43522116,  0.41562327,  0.41814903])

In [31]:
sample_submission['y'] = preds

In [32]:
sample_submission.head(5)

,Num,y
0,348622,3098.119369
1,348623,31132.837802
2,348624,334972.050497
3,348625,40890.440505
4,348626,3098.119369


In [33]:
# In GBM you can get some negative predictions:
print sample_submission[sample_submission['y'] < 0]

Empty DataFrame
Columns: [Num, y]
Index: []


In [34]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [35]:
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)

In [36]:
def smape100(estimator, X, Y):
    pred = estimator.predict(X)
    return 100 * np.sum(2 * np.abs(pred - Y) / (np.abs(pred) + np.abs(Y)))/len(pred)

In [37]:
smape100(model, X, y)

39.853185182566925